<a href="https://colab.research.google.com/github/princetyagitech/NLP/blob/master/RNN_Character_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/Pytorch_course/deep-learning-v2-pytorch/recurrent-neural-networks/char-rnn/data/anna.txt','r') as f:
  text=f.read()

In [ ]:
text[:100]

# Tokenization

In [ ]:
chars=tuple(set(text))
int2char=dict(enumerate(chars))
char2int={ch:ii for ii,ch in int2char.items()}

In [ ]:
import numpy as np
encoded=np.array([char2int[ch] for ch in text])

In [ ]:
encoded

array([50, 38, 80, ..., 74,  4,  0])

In [ ]:
def one_hot_encode(arr, n_labels):
    
    
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
def get_batches(arr, batch_size, seq_length):
   
    
    batch_size_total = batch_size * seq_length
    
    n_batches = len(arr)//batch_size_total
    
    
    arr = arr[:n_batches * batch_size_total]
    
    arr = arr.reshape((batch_size, -1))
    
   
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [ ]:
# check if GPU is available
import torch
import torch.nn as nn
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
      
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
       
        self.dropout = nn.Dropout(drop_prob)
        
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        
       
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
       
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

           
            h = tuple([each.data for each in h])

            
            net.zero_grad()
            
            
            output, h = net(inputs, h)
            
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() 
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 20 
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2595... Val Loss: 3.2228
Epoch: 1/20... Step: 20... Loss: 3.1524... Val Loss: 3.1431
Epoch: 1/20... Step: 30... Loss: 3.1402... Val Loss: 3.1251
Epoch: 1/20... Step: 40... Loss: 3.1095... Val Loss: 3.1180
Epoch: 1/20... Step: 50... Loss: 3.1369... Val Loss: 3.1154
Epoch: 1/20... Step: 60... Loss: 3.1157... Val Loss: 3.1119
Epoch: 1/20... Step: 70... Loss: 3.1005... Val Loss: 3.1055
Epoch: 1/20... Step: 80... Loss: 3.1013... Val Loss: 3.0920
Epoch: 1/20... Step: 90... Loss: 3.0795... Val Loss: 3.0548
Epoch: 1/20... Step: 100... Loss: 2.9958... Val Loss: 2.9736
Epoch: 1/20... Step: 110... Loss: 3.0341... Val Loss: 2.9525
Epoch: 1/20... Step: 120... Loss: 2.8519... Val Loss: 2.8357
Epoch: 1/20... Step: 130... Loss: 2.7529... Val Loss: 2.7110
Epoch: 2/20... Step: 140... Loss: 2.6542... Val Loss: 2.6009
Epoch: 2/20... Step: 150... Loss: 2.5919... Val Loss: 2.5275
Epoch: 2/20... Step: 160... Loss: 2.5187... Val Loss: 2.4837
Epoch: 2/20... Step: 170... Loss:

In [ ]:
def predict(net, char, h=None, top_k=None):
       
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        
        out, h = net(inputs, h)

       
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() 
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() 
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
   
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))